# T-725 Natural Language Processing: Lab 2
In today's lab, we will be working with text classification.

To begin with, do the following:
* Select `"File" > "Save a copy in Drive"` to create a local copy of this notebook that you can edit.
* Select `"Runtime" > "Run all"` to run the code in this notebook.

## List comprehensions in Python
List comprehensions are a concise way of creating lists in Python, and take the form:

```python
[expression for item in iterable]
```

A list comprehension creates a new list by evaluating some expression for every item in a given iterable (such as a string, a list or a dictionary). Let's look at an example:

In [ ]:
sentence = "In a hole in the ground there lived a hobbit."
words = sentence.split()
print(words)

# Example of a list comprehension
word_lengths = [len(word) for word in words]
print(word_lengths)

# This is equal to
word_lengths = []
for word in words:
  word_lengths.append(len(word))

print(word_lengths)

You can also add a conditional statement to list comprehensions, so that the expression will only be evaluated for items that meet a certain criteria:

In [ ]:
e_words = [word for word in words if len(word) > 5]
print(e_words)

Python also has set and dictionary comprehensions:

In [ ]:
lowercase_characters = {c.lower() for c in sentence}
print(lowercase_characters)

word_length = {word: len(word) for word in words}
print(word_length['ground'])

A nested list is a list within another list. You can iterate through nested lists in the following way:

In [ ]:
# A list of countries and their capitals within different continents
continents = [
    [('Iceland', 'Reykjavík'), ('Germany', 'Berlin'), ('Spain', 'Madrid')],  # Europe
    [('Japan', 'Tokyo'), ('China', 'Beijing'), ('South Korea', 'Seoul')],  # Asia
    [('Nigeria', 'Abuja'), ('Algeria', 'Algiers'), ('Angola', 'Luanda')]  # Africa
]

# Create a list of all the countries in the previous list
[country for continent in continents for (country, capital) in continent]

## Sentiment analysis with NLTK
[Chapter 6](https://www.nltk.org/book/ch06.html) of the NLTK book shows how the toolkit can be used to create document classifiers, including a sentiment analyzer. The NLTK includes the `movie_reviews` corpus, which contains 2,000 movie reviews. Half of the reviews have been labelled as **positive** and the other half as **negative**. Let's download it and take a look:

In [ ]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('punkt')

nltk.download('movie_reviews')
print("Categories:", movie_reviews.categories())

As expected, there are two categories: `pos` for positive reviews and `neg` for negative reviews. For this particular corpus, each review is stored as a separate text file. To get a list of all the text files in the corpus, we can use `movie_reviews.fileids()`. We can also get a list of files for a specific category:

In [ ]:
pos_fileids = movie_reviews.fileids('pos')
neg_fileids = movie_reviews.fileids('neg')

print(pos_fileids[:5])  # The first 5 positive reviews
print(neg_fileids[:5])  # The first 5 negative reviews

We can get a list of all the tokens in the corpus with `movie_reviews.words()`. We can also specify a filename to get a single tokenized review:

In [ ]:
pos_reviews = [movie_reviews.words(fid) for fid in pos_fileids]
neg_reviews = [movie_reviews.words(fid) for fid in neg_fileids]

print(pos_reviews[0][:10])  # The first 10 tokens of the first positive review
print(neg_reviews[0][:10])  # The first 10 tokens of the first negative review

Some words, such as *brilliant* and *memorable*, are more strongly associated with positive reviews than negative ones. Similarly, *boring* and *unfunny* have a stronger association with negative reviews.

Using the movie review corpus, we can train a classifier to predict whether a given review is positive or negative. The classifier extracts a set of *features* from every review, which are then used to make the classification. In this case, the features we use will be a dictionary that tells us whether each of the 2,000 most common words in the corpus is present within a review or not.

In [ ]:
# Create a set with 2,000 of the most frequent words in the movie review corpus
movie_fd = nltk.FreqDist(movie_reviews.words())
movie_words = {word for word, count in movie_fd.most_common(2000)}

# For a given review (in the form of a list or set of tokens), create a
# dictionary which tells us which words are present and which are not.
def get_review_features(review):
  review_words = set(review)
  return {word: word in review_words for word in movie_words}

In [ ]:
# Let's see how this works for the first positive review:
example_features = get_review_features(pos_reviews[0])
print("'funny' is in the review:", example_features['funny'])
print("'boring' is in the review:", example_features['boring'])

Next, let's create a training set that we can use to train a Naive Bayesian classifier. The training set, in this case, is a list of tuples in the format `[(features, category), ...]`, where `features` is a dictionary from `get_review_features()` and `category` is either `pos` or `neg`, depending on whether the review is positive or negative. To get an idea of how well the classifier performs, we're going to reserve 10% of the reviews for testing. That means that we'll be training our classifier on 1800 examples and testing it on 200 examples.

In [ ]:
pos_examples = [(get_review_features(review), 'pos') for review in pos_reviews]
neg_examples = [(get_review_features(review), 'neg') for review in neg_reviews]

movie_training = pos_examples[:900] + neg_examples[:900]  # 1800 examples total
movie_test = pos_examples[900:] + neg_examples[900:]  # 200 examples total

Now we have everything we need to train our classifier.

In [ ]:
movie_classifier = nltk.NaiveBayesClassifier.train(movie_training)

How well does it perform on the test set?

In [ ]:
print("Accuracy:", nltk.classify.accuracy(movie_classifier, movie_test))

The classifier achieves an accuracy of 81.5%. Let's take a look at which words have the biggest weights:

In [ ]:
movie_classifier.show_most_informative_features(20)

# Assignment
Answer the following questions and hand in your solution in Canvas before 8:30 AM Monday September 11th. Remember to save your file before uploading it.

## Question 1
The NLTK also includes a `subjectivity` corpus, which contains a collection of sentences that have either been categorized as **subjective** (emotional, expressing personal feelings and views)  or **objective** (more rational, factual). Some examples:

* **Objective sentences**:
  * uma thurman stars in quentin tarantino's fourth film venture , kill bill .  
  * he lives in a motor garage with his six friends .
  * the ensuing battle was one of the most savage in u . s . history .
* **Subjective sentences**:
  * seagal's strenuous attempt at a change in expression could very well clinch him this year's razzie .
  * de niro cries . you'll cry for your money back .
  * a heroic tale of persistence that is sure to win viewers' hearts .

Unlike the movie review corpus, where every review is stored in separate file, here there is only one file for each category.

Complete the following tasks:
1. Import and download the `subjectivity` corpus.
2. Find the names of each category.
3. Using the category names, get the relative path of each file.
4. Get a list of tokenized sentences for each category (using `subjectivity.sents(fileid)`).

In [ ]:
# Your solution here
import nltk
from nltk.corpus import subjectivity

nltk.download('subjectivity')

print("2) Categories:", subjectivity.categories(),"\n")

print("3.1) Relative path of 'obj': ", subjectivity.fileids("obj"))
print("3.2) Relative path of 'subj': ", subjectivity.fileids("subj"),"\n")

sentences_obj = subjectivity.sents(subjectivity.fileids("obj"))
sentences_subj = subjectivity.sents(subjectivity.fileids("subj"))
print("4.1) Tokenized sentences for 'obj': ", sentences_obj)
print("4.2) Tokenized sentences for 'subj': ", sentences_subj)

## Question 2
Complete the following tasks:
1. Create a set with the 2,000 most common words in the `subjectivity` corpus using `nltk.FreqDist()`.
2. Create a function that takes a single, tokenized sentence as input (e.g., `['the', 'ensuing', 'battle', ...]`), and returns a dictionary of the 2,000 most frequent words and whether or not they are in the sentence (e.g., `{'battle': True, 'amusing': False, ...}`).

In [ ]:
# Your solution here
subj_freq_dist = nltk.FreqDist(subjectivity.words())
most_common_2000 = {word for word, count in subj_freq_dist.most_common(2000)}

def get_dictionary(sentence):
    sentence_words = set(sentence)
    dict = {}
    for word in most_common_2000:
        if word in sentence_words:
            dict[word] = True
        else:
            dict[word] = False
    return dict

## Question 3
Complete the following tasks:
1. Create a training set with 9,000 sentences (4,500 of each category)
2. Create a test set with 1,000 sentences (500 of each category)

In [ ]:
# Your solution here
subj_examples = [(get_dictionary(sentence), 'subj') for sentence in subjectivity.sents(subjectivity.fileids("subj"))]
obj_examples = [(get_dictionary(sentence), 'obj') for sentence in subjectivity.sents(subjectivity.fileids("obj"))]

train_data = subj_examples[:4500] + obj_examples[:4500]
test_data = subj_examples[4500:] + obj_examples[4500:]

## Question 4
Complete the following tasks:
1. Train a Naive Bayes classifier using the training set from the previous question.
2. Evaluate the classifier on the test set. How accurate is it?
3. Find the 20 most informative features.

In [ ]:
# Your solution here

subj_obj_classifier = nltk.NaiveBayesClassifier.train(train_data)

print("Accuracy:", nltk.classify.accuracy(subj_obj_classifier, test_data))

subj_obj_classifier.show_most_informative_features(20)


# Question 5
Dialog acts are sort of the type of *action* performed by the speaker. In the instant messaging corpus dataset 'NPS', each utterance is labeled with one of 15 dialogue act types, such as **Statement**, **Emotion**, **ynQuestion**, **Continuer**, etc.

Your task is to classify text from the NPS corpus into two dialog acts: **whQuestion** or **Emotion**.

Start by downloading the NPS corpus and getting all posts from the corpus:

In [37]:
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()

[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


Create a list that only includes posts of class **Emotion** and **whQuestion**. You can access the class of a post by calling `post.get("class")`.

In [38]:
# Your solution here
posts_list = []
for post in posts:
    if post.get("class") == "Emotion" or post.get("class") == "whQuestion":
        posts_list.append(post)
        
print(posts_list[0].text, posts_list[0].get("class"))
print(posts_list[10].text, posts_list[10].get("class"))

:P Emotion
haha Emotion


Randomize the posts and create a training set and a test set, where the first 1300 **Emotion + whQuestion** posts are used for training and the rest for testing.

In [39]:
# Your solution here
import random
random.shuffle(posts_list)

train_set = posts_list[:1300]
test_set = posts_list[1300:]

Create a list of the 200 most frequent tokens in the training set. You can access the text of a `post` object by calling `post.text`. Remember that the **split** function will use whitespace to tokenize a string: `some_string.split()`

In [40]:
# Your solution here
freq_dist = nltk.FreqDist()
for post in train_set:
    freq_dist.update(post.text.split())

most_common_200 = {word for word, count in freq_dist.most_common(200)}


Define two feature selection functions that take a string as input and output a dictionary of features:
* `get_word_features(string)`
* `get_custom_features(string)`

Begin by defining `get_word_features`. This function should use the words as features, just like in the movie review example above.




In [46]:
# Your solution here
def get_word_features(string):
    words = string.split()
    dict = {}
    for word in most_common_200:
        if word in words:
            dict[word] = True
        else:
            dict[word] = False
    return dict

Next, define `get_custom_features`. This function should extract the features from the text that characterize the **Emotion** and **whQuestions** classes.

In [47]:
# Your solution here
def get_custom_features(string):
    words = string.split()
    features = ["who", "what", "when", "where", "why", "how", "!", "?", "lol", "hahaha", ":)", ":(", "lmao", "rofl", "teehee"]
    dict = {}
    for word in words:
        if word in features:
            dict[word] = True
        else:
            dict[word] = False
    return dict

Conduct the following tasks:
*   Train two Naive Bayes classifiers on the **Emotion + whQuestions** training set: one that uses the `get_word_features` function and another using `get_custom_features`.
*   Evaluate each classifier on the test set. How accurate are they? Which one is better?
*   What are the 20 most informative features for each classifier?


In [45]:
# Your solution here

training_set = []
for post in train_set:
    training_set.append((get_word_features(post.text), post.get("class")))
word_classifier = nltk.NaiveBayesClassifier.train(training_set)

training_set = []
for post in train_set:
    training_set.append((get_custom_features(post.text), post.get("class")))
custom_classifier = nltk.NaiveBayesClassifier.train(training_set)

# transform test_set to the same format as training_set
test_set = []
for post in posts_list[1300:]:
    test_set.append((get_word_features(post.text), post.get("class")))
print("Accuracy word_classifier:", nltk.classify.accuracy(word_classifier, test_set))

test_set = []
for post in posts_list[1300:]:
    test_set.append((get_custom_features(post.text), post.get("class")))
print("Accuracy custom_classifier:", nltk.classify.accuracy(custom_classifier, test_set))

word_classifier.show_most_informative_features(20)
custom_classifier.show_most_informative_features(20)


Accuracy word_classifier: 0.9469026548672567
Accuracy custom_classifier: 0.9528023598820059
Most Informative Features
                    what = True           whQues : Emotio =     95.5 : 1.0
                      is = True           whQues : Emotio =     61.5 : 1.0
                     you = True           whQues : Emotio =     40.9 : 1.0
                     and = True           whQues : Emotio =     31.8 : 1.0
                     lol = True           Emotio : whQues =     20.7 : 1.0
                      in = True           whQues : Emotio =     20.5 : 1.0
                    that = True           whQues : Emotio =     20.5 : 1.0
                     the = True           whQues : Emotio =     18.2 : 1.0
                      to = True           whQues : Emotio =     12.2 : 1.0
                      ok = True           whQues : Emotio =     12.0 : 1.0
                      so = True           whQues : Emotio =     11.5 : 1.0
                    lmao = True           Emotio : whQues